In [5]:
!pip install transformers
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer


In [6]:
##### This BERT-large that has already been fine-tuned for the SQuAD benchmark.

bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [19]:
# Now that we have loaded the fine tuned
# model, we are ready to feed questions to it

question = "What is Opera?"
answer_text = "Opera refers to a dramatic art form, originating in Europe, in which the emotional content is conveyed to the audience as much through music, both vocal and instrumental, as it is through the lyrics. By contrast, in musical theater an actor's dramatic performance is primary, and the music plays a lesser role. The drama in opera is presented using the primary elements of theater such as scenery, costumes, and acting. However, the words of the opera, or libretto, are sung rather than spoken. The singers are accompanied by a musical ensemble ranging from a small instrumental ensemble to a full symphonic orchestra"
input_ids = tokenizer.encode(question, answer_text)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
what          2,054
is            2,003
opera         3,850
?             1,029

[SEP]           102

opera         3,850
refers        5,218
to            2,000
a             1,037
dramatic      6,918
art           2,396
form          2,433
,             1,010
originating  14,802
in            1,999
europe        2,885
,             1,010
in            1,999
which         2,029
the           1,996
emotional     6,832
content       4,180
is            2,003
conveyed     21,527
to            2,000
the           1,996
audience      4,378
as            2,004
much          2,172
through       2,083
music         2,189
,             1,010
both          2,119
vocal         5,554
and           1,998
instrumental  6,150
,             1,010
as            2,004
it            2,009
is            2,003
through       2,083
the           1,996
lyrics        4,581
.             1,012
by            2,011
contrast      5,688
,             1,010
in            1,999
musical       3,31

In [20]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [25]:
##### feed the data to the model  ######

start_scores, end_scores = bert_model(torch.tensor([input_ids]),
                                 token_type_ids=torch.tensor([segment_ids]))

# Find the tokens with the highest "start" and "end" scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print("You asked: ", question)
print("The passage was: ", answer_text)
print('The answer to your question is : "' + answer + '"')

### This model can accept random passages and questions from those passages. 
### It works well on this passage which has no connection with the dataset on which the model was trained.

You asked:  What is Opera?
The passage was:  Opera refers to a dramatic art form, originating in Europe, in which the emotional content is conveyed to the audience as much through music, both vocal and instrumental, as it is through the lyrics. By contrast, in musical theater an actor's dramatic performance is primary, and the music plays a lesser role. The drama in opera is presented using the primary elements of theater such as scenery, costumes, and acting. However, the words of the opera, or libretto, are sung rather than spoken. The singers are accompanied by a musical ensemble ranging from a small instrumental ensemble to a full symphonic orchestra
The answer to your question is : "a dramatic art form"
